In [2]:
import numpy as np
import pandas as pd
import math
import sys

# ==============================================================================
# CÁC HÀM CÔNG CỤ
# ==============================================================================

def calculate_forward_differences(y_values):
    """Tạo bảng sai phân tiến đầy đủ."""
    n = len(y_values)
    diff_table = np.zeros((n, n))
    diff_table[:, 0] = y_values
    for j in range(1, n):
        for i in range(n - j):
            diff_table[i, j] = diff_table[i+1, j-1] - diff_table[i, j-1]
    return diff_table

def get_standard_form_from_newton(div_diff_coeffs, x_nodes):
    """Chuyển đa thức Newton P(x) sang dạng chính tắc A₀ + A₁x + ..."""
    n = len(div_diff_coeffs)
    poly_coeffs = np.array([div_diff_coeffs[n-1]])
    for i in range(n - 2, -1, -1):
        poly_coeffs = np.polymul(poly_coeffs, [1, -x_nodes[i]])
        poly_coeffs[-1] += div_diff_coeffs[i]
    return np.flip(poly_coeffs)

def format_polynomial(coeffs, var_name='t'):
    """Tạo chuỗi biểu diễn đa thức từ mảng hệ số [c₀, c₁, c₂, ...]."""
    if np.all(np.abs(coeffs) < 1e-12): return "0.0"
    
    poly_str = []
    if abs(coeffs[0]) > 1e-12:
        poly_str.append(f"{coeffs[0]:.8f}")
        
    for i in range(1, len(coeffs)):
        if abs(coeffs[i]) > 1e-12:
            sign = " + " if coeffs[i] > 0 else " - "
            coeff_val = abs(coeffs[i])
            
            if i == 1:
                poly_str.append(f"{sign}{coeff_val:.8f}*{var_name}")
            else:
                poly_str.append(f"{sign}{coeff_val:.8f}*{var_name}^{i}")
    
    result = "".join(poly_str).strip()
    if result.startswith("+ "):
        result = result[2:]
    return result

# ==============================================================================
# BƯỚC 0: THIẾT LẬP, ĐỌC DỮ LIỆU VÀ CHỌN MỐC TỰ ĐỘNG
# ==============================================================================

np.set_printoptions(suppress=True, precision=8)
pd.options.display.float_format = '{:.8f}'.format

print("=======================================================================")
print("BƯỚC 0: ĐỌC DỮ LIỆU VÀ CHỌN MỐC TỰ ĐỘNG")
print("=======================================================================")

file_path = "data/input_newton_moc_cach_deu.xlsx"
try:
    df = pd.read_excel(file_path)
    x_full, y_full = df['x'].to_numpy(), df['y'].to_numpy()
    print(f"Đã đọc thành công {len(df)} dòng dữ liệu từ file '{file_path}'")
except (FileNotFoundError, KeyError) as e:
    print(f"LỖI: Không thể đọc file hoặc thiếu cột 'x'/'y'. Chi tiết: {e}"); sys.exit()

h = x_full[1] - x_full[0]
print(f"Bước nhảy h được tính tự động: {h:.8f}")

# --- NHẬP DỮ LIỆU TỪ BÀN PHÍM ---
while True:
    try:
        num_points_interpolation = int(input(f"Nhập số điểm nội suy (ví dụ: 7): "))
        if 2 <= num_points_interpolation < len(x_full):
            break
        else:
            print(f"Số điểm không hợp lệ. Phải nằm trong khoảng [2, {len(x_full)-1}].")
    except ValueError:
        print("Vui lòng nhập một số nguyên.")

while True:
    try:
        x_target = float(input("Nhập giá trị x cần tính toán (ví dụ: 4.14661): "))
        break
    except ValueError:
        print("Vui lòng nhập một số thực.")

# --- TỰ ĐỘNG CHỌN MỐC TỐI ƯU ---
closest_index = np.argmin(np.abs(x_full - x_target))
start_index = closest_index - (num_points_interpolation // 2)
start_index = max(0, start_index)
start_index = min(start_index, len(x_full) - num_points_interpolation)

print("\n--- Tự động chọn mốc nội suy ---")
print(f"Giá trị x cần tính {x_target} gần nhất với x[{closest_index}] = {x_full[closest_index]:.4f}")
print(f"Đã tự động chọn {num_points_interpolation} điểm bắt đầu từ chỉ số: {start_index}")

# Lấy dữ liệu con để tính toán
x_sub = x_full[start_index : start_index + num_points_interpolation]
y_sub = y_full[start_index : start_index + num_points_interpolation]

print(f"\nCác giá trị (x, y) được dùng để tính toán (từ {start_index} đến {start_index + num_points_interpolation - 1}):")
df_selected = pd.DataFrame({'x': x_sub, 'y': y_sub}, index=np.arange(start_index, start_index + num_points_interpolation))
print(df_selected.to_string())

# ==============================================================================
# BƯỚC 1: BẢNG SAI PHÂN TIẾN ĐẦY ĐỦ
# ==============================================================================
print("\n\n=======================================================================")
print("BƯỚC 1: BẢNG SAI PHÂN TIẾN ĐẦY ĐỦ")
print("=======================================================================")
diff_table = calculate_forward_differences(y_sub) 
df_display = pd.DataFrame(index=np.arange(start_index, start_index + len(x_sub)))
df_display['x'] = x_sub
df_display['y'] = y_sub
for j in range(1, len(x_sub)):
    col_name = f'Δ^{j}y'
    new_col = np.full(len(x_sub), np.nan)
    # --- SỬA ĐỔI: Lấy dữ liệu từ diff_table ---
    valid_data = diff_table[0 : len(x_sub) - j, j]
    new_col[j:] = valid_data
    df_display[col_name] = new_col
print(df_display.to_string())

# ==============================================================================
# BƯỚC 2: CÁC HÀNG HỆ SỐ (D_tiến và Dy)
# ==============================================================================
print("\n\n=======================================================================")
print("BƯỚC 2: CÁC HÀNG HỆ SỐ (D_tiến và Dy)")
print("=======================================================================")
coeffs_D_tien = diff_table[0, :]
coeffs_Dy_newton = np.array([diff_table[0, k] / math.factorial(k) for k in range(num_points_interpolation)])
df_dy_all = pd.DataFrame([coeffs_D_tien, coeffs_Dy_newton], index=['D_tiến', 'Dy'])
print(df_dy_all.to_string(header=False))

# ==============================================================================
# BƯỚC 3: BẢNG TÍCH VÀ ĐA THỨC NỘI SUY P(t)
# ==============================================================================
print("\n\n=======================================================================")
print('BƯỚC 3: BẢNG TÍCH VÀ ĐA THỨC NỘI SUY P(t)')
print("=======================================================================")
print("Bảng Tích chứa hệ số của P_k(t) = t(t-1)...(t-k+1)")
rows = num_points_interpolation
cols = num_points_interpolation
product_table_coeffs = np.zeros((rows, cols), dtype=int)
current_poly = np.array([1])
for k in range(rows):
    start_col = cols - len(current_poly)
    product_table_coeffs[k, start_col:] = current_poly
    current_poly = np.polymul(current_poly, [1, -k])
df_product = pd.DataFrame(product_table_coeffs, index=np.arange(rows))
print(df_product.to_string(header=False))

# --- TÍNH VÀ IN RA ĐA THỨC P(t) ---
coeffs_poly_t_standard = np.zeros(num_points_interpolation)
for k in range(num_points_interpolation):
    term_poly_coeffs = np.flip(product_table_coeffs[k, :])
    term = coeffs_Dy_newton[k] * term_poly_coeffs
    coeffs_poly_t_standard += term

print("\nĐa thức nội suy P(t) dạng chính tắc:")
print(f"P(t) = {format_polynomial(coeffs_poly_t_standard, 't')}")


# ==============================================================================
# BƯỚC 4: HỆ SỐ CỦA ĐA THỨC NỘI SUY P(x)
# ==============================================================================
print("\n\n=======================================================================")
print(f"BƯỚC 4: HỆ SỐ CỦA ĐA THỨC NỘI SUY P(x) DẠNG CHÍNH TẮC")
div_diff_coeffs = np.zeros(num_points_interpolation)
for k in range(num_points_interpolation):
    div_diff_coeffs[k] = full_diff_table[0, k] / (math.factorial(k) * (h**k))
standard_coeffs_poly_x = get_standard_form_from_newton(div_diff_coeffs, x_sub_interpolation)
print(f"P(x) = {format_polynomial(standard_coeffs_poly_x, 'x')}")

# ==============================================================================
# BƯỚC 5: TÍNH TOÁN CUỐI CÙNG VÀ KIỂM TRA
# ==============================================================================
print("\n\n=======================================================================")
print(f"BƯỚC 5: TÍNH TOÁN TẠI x = {x_target} VÀ KIỂM TRA")
print("=======================================================================")
x0 = x_full[start_index]
t = (x_target - x0) / h
print(f"Giá trị t tương ứng: t = ({x_target} - {x0}) / {h} = {t:.8f}")

# --- PHƯƠNG PHÁP 1: TÍNH TOÁN DỰA TRÊN ĐA THỨC P(t) ---
print("\n--- Phương pháp 1: Tính toán dựa trên đa thức P(t) ---")
p_t_val = np.polyval(np.flip(coeffs_poly_t_standard), t)
poly_t_prime = np.polyder(np.flip(coeffs_poly_t_standard), 1)
p_t_prime_val = np.polyval(poly_t_prime, t)
poly_t_double_prime = np.polyder(np.flip(coeffs_poly_t_standard), 2)
p_t_double_prime_val = np.polyval(poly_t_double_prime, t)

p_x_val_from_t = p_t_val
p_prime_x_val_from_t = p_t_prime_val / h
p_double_prime_x_val_from_t = p_t_double_prime_val / (h**2)

print(f"P(x)   = P(t)                               = {p_x_val_from_t:.8f}")
print(f"P'(x)  = P'(t) * (1/h) = {p_t_prime_val:.8f} / {h} = {p_prime_x_val_from_t:.8f}")
print(f"P''(x) = P''(t) * (1/h²) = {p_t_double_prime_val:.8f} / {h**2} = {p_double_prime_x_val_from_t:.8f}")


# --- PHƯƠNG PHÁP 2: TÍNH TOÁN DỰA TRÊN ĐA THỨC P(x) ĐỂ KIỂM TRA ---
print("\n--- Phương pháp 2: Tính toán dựa trên đa thức P(x) để kiểm tra ---")
p_x_val = np.polyval(np.flip(standard_coeffs_poly_x), x_target)
poly_prime = np.polyder(np.flip(standard_coeffs_poly_x), 1)
p_prime_x_val = np.polyval(poly_prime, x_target)
poly_double_prime = np.polyder(np.flip(standard_coeffs_poly_x), 2)
p_double_prime_x_val = np.polyval(poly_double_prime, x_target)

print(f"P(x)   tại {x_target} = {p_x_val:.8f}")
print(f"P'(x)  tại {x_target} = {p_prime_x_val:.8f}")
print(f"P''(x) tại {x_target} = {p_double_prime_x_val:.8f}")

BƯỚC 0: ĐỌC DỮ LIỆU VÀ CHỌN MỐC TỰ ĐỘNG
Đã đọc thành công 201 dòng dữ liệu từ file 'data/input_newton_moc_cach_deu.xlsx'
Bước nhảy h được tính tự động: 0.11800000


Nhập số điểm nội suy (ví dụ: 7):  7
Nhập giá trị x cần tính toán (ví dụ: 4.14661):  4.12



--- Tự động chọn mốc nội suy ---
Giá trị x cần tính 4.12 gần nhất với x[26] = 4.0680
Đã tự động chọn 7 điểm bắt đầu từ chỉ số: 23

Các giá trị (x, y) được dùng để tính toán (từ 23 đến 29):
            x          y
23 3.71400000 2.82670000
24 3.83200000 2.43840000
25 3.95000000 2.03540000
26 4.06800000 1.62340000
27 4.18600000 1.20840000
28 4.30400000 0.79620000
29 4.42200000 0.39280000


BƯỚC 1: BẢNG SAI PHÂN TIẾN ĐẦY ĐỦ
            x          y        Δ^1y        Δ^2y       Δ^3y        Δ^4y        Δ^5y       Δ^6y
23 3.71400000 2.82670000         NaN         NaN        NaN         NaN         NaN        NaN
24 3.83200000 2.43840000 -0.38830000         NaN        NaN         NaN         NaN        NaN
25 3.95000000 2.03540000 -0.40300000 -0.01470000        NaN         NaN         NaN        NaN
26 4.06800000 1.62340000 -0.41200000 -0.00900000 0.00570000         NaN         NaN        NaN
27 4.18600000 1.20840000 -0.41500000 -0.00300000 0.00600000  0.00030000         NaN        NaN
28 4

NameError: name 'full_diff_table' is not defined